In [12]:

import pandas as pd
import numpy as np


In [13]:
df = pd.read_csv("Scooby-Doo Completed.csv")

df.head()

,index,series.name,network,season,title,imdb,engagement,date.aired,run.time,format,...,batman,scooby-dum,scrappy-doo,hex.girls,blue.falcon,fred.va,daphnie.va,velma.va,shaggy.va,scooby.va
0,1,"Scooby Doo, Where Are You!",CBS,1,What a Night for a Knight,8.1,556.0,1969-09-13,21,TV Series,...,False,False,False,False,False,Frank Welker,Stefanianna Christopherson,Nicole Jaffe,Casey Kasem,Don Messick
1,2,"Scooby Doo, Where Are You!",CBS,1,A Clue for Scooby Doo,8.1,479.0,1969-09-20,22,TV Series,...,False,False,False,False,False,Frank Welker,Stefanianna Christopherson,Nicole Jaffe,Casey Kasem,Don Messick
2,3,"Scooby Doo, Where Are You!",CBS,1,Hassle in the Castle,8.0,455.0,1969-09-27,21,TV Series,...,False,False,False,False,False,Frank Welker,Stefanianna Christopherson,Nicole Jaffe,Casey Kasem,Don Messick
3,4,"Scooby Doo, Where Are You!",CBS,1,Mine Your Own Business,7.8,426.0,1969-10-04,21,TV Series,...,False,False,False,False,False,Frank Welker,Stefanianna Christopherson,Nicole Jaffe,Casey Kasem,Don Messick
4,5,"Scooby Doo, Where Are You!",CBS,1,Decoy for a Dognapper,7.5,391.0,1969-10-11,21,TV Series,...,False,False,False,False,False,Frank Welker,Stefanianna Christopherson,Nicole Jaffe,Casey Kasem,Don Messick


In [ ]:
def get_caught_by(row):
    return_character=[]
    for character in ['fred', 'daphnie', 'velma', 'shaggy', 'scooby']:
        if row[f'caught.{character}']:
            return_character.append(character.capitalize())
    if len(return_character)==0:
        return_character.append("None")        
    return return_character

In [ ]:
def unmasked_by(row):
    return_character=[]
    for character in ['fred', 'daphnie', 'velma', 'shaggy', 'scooby']:
        if row[f'unmask.{character}']:
            return_character.append(character.capitalize())
    if len(return_character)==0:
        return_character.append("None")        
    return return_character


In [ ]:
# Function to handle complex replacements in 'monster.type'
def replace_monster_type(monster_type):
    # Split the entry by commas, strip each element, and replace terms
    monster_list = [x.strip() for x in monster_type.split(',')]
    
    # Replace 'Disguise' and variants with 'Disguised'
    monster_list = ['Disguised' if term in ['Disguise', 'Disugised', ' Disguised'] else term for term in monster_list]
    
    # Join the list back into a single string
    return ','.join(monster_list)

# Apply the function to replace terms in 'monster.type'

df['monster.type'] = df['monster.type'].replace('NULL', 'None').fillna('None')

df['monster.type'] = df['monster.type'].apply(replace_monster_type)

# Now you can continue with the rest of your code as it is
df['monster.name'] = df['monster.name'].replace('NULL', 'None').fillna('None')

df['monster.real'] = df['monster.real'].replace('NULL', 'None').fillna('None')
df['monster.subtype'] = df['monster.subtype'].replace('NULL', 'None').fillna('None')
df['monster.gender'] = df['monster.gender'].replace('NULL', 'None').fillna('None')
df['monster.species'] = df['monster.species'].replace('NULL', 'None').fillna('None')

df['caught.fred'] = df['caught.fred'].replace('NULL', 'False').fillna('None')
df['caught.daphnie'] = df['caught.daphnie'].replace('NULL', 'False').fillna('None')
df['caught.velma'] = df['caught.velma'].replace('NULL', 'False').fillna('None')
df['caught.shaggy']= df['caught.shaggy'].replace('NULL', 'False').fillna('None')
df['caught.scooby']= df['caught.scooby'].replace('NULL', 'False').fillna('None')

df['captured.fred'] = df['captured.fred'].replace('NULL', 'False').fillna('None')
df['captured.daphnie'] = df['captured.daphnie'].replace('NULL', 'False').fillna('None')
df['captured.velma'] = df['captured.velma'].replace('NULL', 'False').fillna('None')
df['captured.shaggy']= df['captured.shaggy'].replace('NULL', 'False').fillna('None')
df['captured.scooby']= df['captured.scooby'].replace('NULL', 'False').fillna('None')

df['fred.va'] = df['fred.va'].ffill()
df['daphnie.va'] = df['daphnie.va'].ffill()
df['velma.va'] = df['velma.va'].ffill()
df['shaggy.va']= df['shaggy.va'].ffill()
df['scooby.va']= df['scooby.va'].ffill()

df['monster_caught_by'] = df.apply(get_caught_by, axis=1)
df['monster_unmasked_by'] = df.apply(unmasked_by, axis=1)


df.interpolate(method ='linear', limit_direction ='forward', inplace=True)

# Preview the flattened dataset after processing

# You can continue your analysis here


C:\Users\sanad\AppData\Local\Temp\ipykernel_43628\277461643.py:48: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method ='linear', limit_direction ='forward', inplace=True)


,monster.type,monster.gender
0,Possessed Object,Male
1,Ghost,Male
2,Ghost,Male
3,Ancient,Male
4,Ancient,Female
5,Ghost,Male
6,Animal,Male
7,Mechanical,Male
8,Ghost,Male
9,Ghost,Male


In [17]:
df.to_csv("ScoobyDooCleaned.csv",index=False)